In [1]:
import pandas as pd

In [23]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import seaborn as sns
import pandas as pd
%matplotlib inline
sns.set_style('darkgrid')

In [13]:
df = pd.read_csv('Opt_CL_SELL.csv')
df.head()

,Profit,HH,Input,Score,ADX_Main,ADX_SMA,Dist_ADX,Dist_Step,H1,L1,...,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23
0,1,12,0.998822,0.512558,2.511652,-11.777973,2.617003,-0.501280,100.007512,99.767127,...,0,0,0,0,0,1,0,0,1,1
1,0,13,1.002178,0.521715,4.342947,21.784856,3.361349,-0.079454,100.338371,99.947364,...,0,1,0,1,0,0,1,1,1,0
2,1,14,1.001501,0.503659,0.731736,15.008255,1.159487,-0.050202,100.353038,99.939908,...,0,1,0,0,0,0,1,0,1,0
3,0,16,0.999824,0.456104,-8.779133,-1.756984,0.203503,-0.599488,100.105342,99.668924,...,1,0,1,0,0,1,1,0,0,1
4,0,17,1.001104,0.527604,5.520721,11.039743,0.313343,-0.454135,100.022543,99.474001,...,1,0,1,0,0,0,1,0,1,1


In [14]:
x=df.iloc[:,1:].values
y=df.iloc[:,0].values

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.20, random_state = 0)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(4839, 40)
(1210, 40)
(4839,)
(1210,)


In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense



classifier = Sequential()

classifier.add(Dense(units= 40, kernel_initializer = 'uniform', activation = 'relu', input_dim = 40))
classifier.add(Dense(units= 30, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units= 20, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units= 15, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units= 10, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units= 5, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units= 3, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(x_train, y_train, batch_size = 5, epochs = 100)


Epoch 1/100
968/968 [==============================] - 2s 1ms/step - loss: 0.6895 - accuracy: 0.5545
Epoch 2/100
968/968 [==============================] - 1s 1ms/step - loss: 0.6874 - accuracy: 0.5551
Epoch 3/100
968/968 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5551: 0s - loss: 0.688
Epoch 4/100
968/968 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5551
Epoch 5/100
968/968 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5551
Epoch 6/100
968/968 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5551
Epoch 7/100
968/968 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5551
Epoch 8/100
968/968 [==============================] - 1s 1ms/step - loss: 0.6872 - accuracy: 0.5551
Epoch 9/100
968/968 [==============================] - 1s 2ms/step - loss: 0.6871 - accuracy: 0.5551
Epoch 10/100
968/968 [==============================] - 2s 2ms/step - los

In [30]:
y_pred_test = classifier.predict(x_test)
y_pred_test=y_pred_test>0.5
import numpy as np
from sklearn import metrics
y_pred_train = classifier.predict(x_train)
y_pred_train=y_pred_train>0.5

print("Accuracy on Train Set ==> ",metrics.accuracy_score(y_train,y_pred_train))
print("Accuracy on Test Set ==> ",metrics.accuracy_score(y_test,y_pred_test))

Accuracy on Train Set ==>  0.5550733622649308
Accuracy on Test Set ==>  0.571900826446281


In [31]:
 classifier.predict(x_test)

array([[0.44356307],
       [0.44356307],
       [0.44356307],
       ...,
       [0.44356307],
       [0.44356307],
       [0.44356307]], dtype=float32)

In [32]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

param_grid = {'n_estimators':[200,500,1000],
              'max_depth':[2,3,4,5],
              'min_samples_leaf':[0.2,0.4,0.6,0.8,1],
              'max_features':['auto','sqrt'],
              'criterion':['gini','entropy']}

grid_rfc = RandomizedSearchCV(rfc,param_grid,n_iter=20,scoring='accuracy',cv = 10,refit = True)

In [33]:
grid_rfc.fit(x_train,y_train)
print("Best Score ==> ", grid_rfc.best_score_)
print("Tuned Paramerers ==> ",grid_rfc.best_params_)
print("Accuracy on Train set ==> ", grid_rfc.score(x_train,y_train))
print("Accuracy on Test set ==> ", grid_rfc.score(x_test,y_test))

Best Score ==>  0.5612725219444588
Tuned Paramerers ==>  {'n_estimators': 1000, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 4, 'criterion': 'entropy'}
Accuracy on Train set ==>  0.5935110560033064
Accuracy on Test set ==>  0.5785123966942148


In [38]:
grid_rfc.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)